# MultiOutputRegressor

test the performance of the classical ML algorithms on the CleanRoom prediction problem

**baseline**: predict the mean value of the test data

**Linear Models**:

1. LinearRegression: simple
2. Ridge: 使用 L2 正则化的线性回归模型，可以减小模型的复杂度。
3. Lasso：使用 L1 正则化的线性回归模型，可以增加稀疏性，适用于特征选择。
4. ElasticNet：结合 L1 和 L2 正则化的线性回归模型，综合了 Ridge 和 Lasso 的优势。

**K-neighbors**：（K 最近邻回归）根据最接近的 K 个邻居的目标值来进行回归预测。

**SVM**: SVR（支持向量回归）：使用支持向量机的回归版本，通过找到一个最优超平面来建立回归模型。

**MLP**：（多层感知器）使用多层神经网络进行回归任务。

**GaussianProcessRegressor**：（高斯过程回归）使用高斯过程建立回归模型，可以估计预测的不确定性。

**Tree Models**:

 1. DecisionTreeRegressor：基于决策树的回归模型，通过划分特征空间来建立回归模型。
 2. RandomForestRegressor（随机森林回归）：基于决策树的集成学习方法，
         通过随机特征选择和样本采样来建立回归模型。
 3. AdaBoostRegressor：基于自适应增强的回归模型，通过迭代训练弱回归模型来建立强大的回归模型。
 4. GradientBoostingRegressor（梯度提升回归）：基于梯度提升树的集成学习方法，
         通过迭代训练弱回归模型来建立强大的回归模型。
 5. BaggingRegressor: 基于袋装方法的回归模型，通过自助采样和平均预测结果来建立回归模型。
 6. ExtraTreesRegressor（极端随机森林回归）：在随机森林中进一步随机化特征和划分点的回归模型。
 7. HistGradientBoostingRegressor（直方图梯度提升回归）：基于直方图加速的梯度提升回归模型，具有更快的训练速度。
 8. XGBRegressor：基于梯度提升树的回归模型，具有高效的训练和预测性能。
 9. LightGBMRegressor：基于梯度提升树的回归模型，具有快速训练速度和高准确性。
 10. CatBoostRegressor: 基于梯度提升树的回归模型，具有处理分类和回归任务的能力。


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import RepeatedKFold, cross_validate, GridSearchCV, KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

from utils.metrics import calculate_metrics, get_ccp_scoring, print_results_table
from utils.datasets import load_and_split_data

In [2]:
!pwd

/workspace/src/RDCVC


In [3]:
# 设置显示中文字体
from pylab import mpl

mpl.rcParams["font.sans-serif"] = ["SimHei"]
# 设置正常显示符号
mpl.rcParams["axes.unicode_minus"] = False

## Data

### read data

In [3]:
data_path = "../../data/processed/rdc_data_cleaned.csv"
x_train, x_test, y_train, y_test = load_and_split_data(
    data_path, test_size=0.1, random_state=42
)

# 打印划分后的数据集大小
print("训练集大小:", len(x_train))
print("测试集大小:", len(x_test))

训练集大小: 350
测试集大小: 39


# Native Supported

## LinearRegression

### Normal LR

In [114]:
model_lr = LinearRegression().fit(x_train, y_train)
calculate_metrics(model_lr.predict(x_test), y_test, print_metrics=True, title="lr")
model_lr = LinearRegression()

# ------------------------ 5 fold ------------------------ #
cv_mthd = RepeatedKFold(n_splits=10, n_repeats=10, random_state=42)
ccp_scoring = get_ccp_scoring()
results = cross_validate(
    model_lr, X=x_train, y=y_train, cv=cv_mthd, scoring=ccp_scoring
)

print_results_table(results, ccp_scoring, title="lr-cv",show_time_cost=True)

                    lr                    
┏━━━━━━━━━┯━━━━━━━━━┯━━━━━━━━━┯━━━━━━━━━━┓
┃ Feature │ MAE(Pa) │  MAPE   │ RMSE(Pa) ┃
┠─────────┼─────────┼─────────┼──────────┨
┃ Airflow │  83.55  │  9.21%  │  135.93  ┃
┃ Pres    │  6.66   │ 708.61% │  10.05   ┃
┗━━━━━━━━━┷━━━━━━━━━┷━━━━━━━━━┷━━━━━━━━━━┛

                                    lr-cv                                    
┏━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┓
┃ Metric       ┃ Mean    ┃ Fold 1  ┃ Fold 2  ┃ Fold 3  ┃ Fold 4   ┃ Fold 5  ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━┩
│ score        │ -600.36 │ -567.30 │ -565.50 │ -523.35 │ -1056.83 │ -426.55 │
│ airflow_mae  │ -74.10  │ -69.80  │ -71.98  │ -72.06  │ -91.83   │ -68.08  │
│ airflow_rmse │ -100.97 │ -98.54  │ -101.33 │ -94.28  │ -138.57  │ -86.19  │
│ pres_mae     │ -6.44   │ -6.11   │ -5.34   │ -7.08   │ -7.85    │ -6.43   │
│ pres_rmse    │ -8.65   │ -8.01   │ -8.06   │ -9.28   │ -11.89   │ -8.05   │
└──────────────┴─────────┴─────────┴─────────┴─────────┴──────────┴─────────┘

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Mean Fit Time        ┃ Mean Score Time       ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━┩
│ 0.001191418170928955 │ 0.0010930919647216797 │
└──────────────────────┴───────────────────────┘

### Ridge

In [67]:
model_lr_ridge = Ridge(alpha=0.001).fit(x_train, y_train)
calculate_metrics(
    model_lr_ridge.predict(x_test), y_test, print_metrics=True, title="lr_ridge"
)

model_lr_ridge = Ridge(alpha=999)
cv_mthd = RepeatedKFold(n_splits=10, n_repeats=10, random_state=42)
ccp_scoring = get_ccp_scoring()
results = cross_validate(
    model_lr_ridge, X=x_train, y=y_train, cv=cv_mthd, scoring=ccp_scoring
)

print_results_table(results, ccp_scoring, title="lr_ridge-cv")

                 lr_ridge                 
┏━━━━━━━━━┯━━━━━━━━━┯━━━━━━━━━┯━━━━━━━━━━┓
┃ Feature │ MAE(Pa) │  MAPE   │ RMSE(Pa) ┃
┠─────────┼─────────┼─────────┼──────────┨
┃ Airflow │  83.55  │  9.21%  │  135.93  ┃
┃ Pres    │  6.66   │ 708.61% │  10.05   ┃
┗━━━━━━━━━┷━━━━━━━━━┷━━━━━━━━━┷━━━━━━━━━━┛

                                 lr_ridge-cv                                 
┏━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┓
┃ Metric       ┃ Mean    ┃ Fold 1  ┃ Fold 2  ┃ Fold 3  ┃ Fold 4   ┃ Fold 5  ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━┩
│ score        │ -709.19 │ -593.40 │ -731.68 │ -642.61 │ -1014.26 │ -523.44 │
│ airflow_mae  │ -83.62  │ -73.33  │ -85.51  │ -82.49  │ -95.87   │ -75.10  │
│ airflow_rmse │ -111.09 │ -100.30 │ -115.52 │ -103.54 │ -135.88  │ -95.99  │
│ pres_mae     │ -6.76   │ -6.52   │ -5.75   │ -7.31   │ -7.72    │ -6.84   │
│ pres_rmse    │ -9.01   │ -8.63   │ -8.06   │ -9.63   │ -11.36   │ -8.74   │
└──────────────┴─────────┴─────────┴─────────┴─────────┴──────────┴─────────┘

### Lasso

In [71]:
model_lr_lasso = Lasso(alpha=0).fit(x_train, y_train)
calculate_metrics(
    model_lr_lasso.predict(x_test), y_test, print_metrics=True, title="lr_lasso"
)
model_lr_lasso = Lasso(alpha=1.0)
cv_mthd = RepeatedKFold(n_splits=5, n_repeats=3, random_state=44)
ccp_scoring = get_ccp_scoring()
results = cross_validate(
    model_lr_lasso, X=x_train, y=y_train, cv=cv_mthd, scoring=ccp_scoring
)

print_results_table(results, ccp_scoring, title="lr_lasso-cv")

/tmp/ipykernel_1310064/1159988914.py:1: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  model_lr_lasso = Lasso(alpha=0).fit(x_train, y_train)
/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.097e+05, tolerance: 3.643e+03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/usr/local/l

                 lr_lasso                 
┏━━━━━━━━━┯━━━━━━━━━┯━━━━━━━━━┯━━━━━━━━━━┓
┃ Feature │ MAE(Pa) │  MAPE   │ RMSE(Pa) ┃
┠─────────┼─────────┼─────────┼──────────┨
┃ Airflow │  83.55  │  9.21%  │  135.93  ┃
┃ Pres    │  6.66   │ 708.61% │  10.05   ┃
┗━━━━━━━━━┷━━━━━━━━━┷━━━━━━━━━┷━━━━━━━━━━┛

                                lr_lasso-cv                                 
┏━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┓
┃ Metric       ┃ Mean    ┃ Fold 1  ┃ Fold 2  ┃ Fold 3  ┃ Fold 4  ┃ Fold 5  ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━┩
│ score        │ -609.73 │ -533.02 │ -790.62 │ -468.04 │ -634.13 │ -587.43 │
│ airflow_mae  │ -74.65  │ -69.22  │ -76.99  │ -69.11  │ -81.68  │ -75.29  │
│ airflow_rmse │ -102.74 │ -95.81  │ -118.44 │ -89.09  │ -104.41 │ -103.36 │
│ pres_mae     │ -6.56   │ -6.31   │ -6.87   │ -6.51   │ -6.31   │ -6.80   │
│ pres_rmse    │ -8.94   │ -9.03   │ -10.21  │ -8.33   │ -8.07   │ -9.08   │
└──────────────┴─────────┴─────────┴─────────┴─────────┴─────────┴─────────┘

### ElasticNet

In [72]:
model_lr_ElasticNet = ElasticNet(alpha=1.0, l1_ratio=0.5).fit(x_train, y_train)
calculate_metrics(
    model_lr_ElasticNet.predict(x_test),
    y_test,
    print_metrics=True,
    title="lr_ElasticNet",
)
model_lr_ElasticNet = ElasticNet(alpha=1.0, l1_ratio=0.5)
cv_mthd = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)
ccp_scoring = get_ccp_scoring()
results = cross_validate(
    model_lr_ElasticNet, X=x_train, y=y_train, cv=cv_mthd, scoring=ccp_scoring
)

print_results_table(results, ccp_scoring, title="lr_ElasticNet-cv")

              lr_ElasticNet               
┏━━━━━━━━━┯━━━━━━━━━┯━━━━━━━━━┯━━━━━━━━━━┓
┃ Feature │ MAE(Pa) │  MAPE   │ RMSE(Pa) ┃
┠─────────┼─────────┼─────────┼──────────┨
┃ Airflow │  83.19  │  9.58%  │  132.61  ┃
┃ Pres    │  6.67   │ 737.32% │   9.93   ┃
┗━━━━━━━━━┷━━━━━━━━━┷━━━━━━━━━┷━━━━━━━━━━┛

                              lr_ElasticNet-cv                              
┏━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┓
┃ Metric       ┃ Mean    ┃ Fold 1  ┃ Fold 2  ┃ Fold 3  ┃ Fold 4  ┃ Fold 5  ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━┩
│ score        │ -611.37 │ -568.99 │ -803.37 │ -696.20 │ -480.34 │ -599.40 │
│ airflow_mae  │ -74.99  │ -71.05  │ -82.55  │ -77.00  │ -70.25  │ -79.04  │
│ airflow_rmse │ -103.11 │ -100.58 │ -119.73 │ -109.58 │ -89.92  │ -103.97 │
│ pres_mae     │ -6.56   │ -5.79   │ -7.45   │ -6.94   │ -6.66   │ -5.91   │
│ pres_rmse    │ -8.88   │ -8.07   │ -10.78  │ -9.36   │ -8.46   │ -7.50   │
└──────────────┴─────────┴─────────┴─────────┴─────────┴─────────┴─────────┘

In [ ]:
import os

# os.chdir("/root/workspace/CCP/")

### Grid for all LR

In [116]:
lr_param_grid = {
    "LinearRegression": {},
    # "Ridge": {"alpha": np.linspace(14, 15, 20 + 2)[1:-1]},
    "Ridge": {"alpha": [0]},
    # "Lasso": {"alpha": np.linspace(0, 100, 100 + 2)[1:-1]},
    "Lasso": {"alpha": [0.09]},
    "ElasticNet": {
        "alpha": np.linspace(0, 10, 10 + 2)[1:-1],
        "l1_ratio": np.linspace(0, 1, 10 + 2)[1:-1],  # 0.89
    },
}

lr_models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(max_iter=10000),
    "Lasso": Lasso(max_iter=1000000),
    "ElasticNet": ElasticNet(fit_intercept=True, max_iter=1000000),
}

for model_name, model in lr_models.items():
    print("=" * 47)
    print(f"Training {model_name}... ")
    print("-" * 80)
    grid_search = GridSearchCV(
        model,
        param_grid=lr_param_grid[model_name],
        scoring=get_ccp_scoring(),
        refit="pres_rmse",
        cv=KFold(n_splits=5, shuffle=True, random_state=42),
        verbose=1,
        n_jobs=1,
    )
    grid_search.fit(x_train, y_train)
    calculate_metrics(
        grid_search.best_estimator_.predict(x_test),
        y_test,
        print_metrics=True,
        title=model_name,
    )
    print("Best Parameters for", model_name, ":", grid_search.best_params_)
    print("Best Score for", model_name, ":", grid_search.best_score_)
    print("=" * 47 + "\n" * 2)

Training LinearRegression... 
--------------------------------------------------------------------------------
Fitting 5 folds for each of 1 candidates, totalling 5 fits


             LinearRegression             
┏━━━━━━━━━┯━━━━━━━━━┯━━━━━━━━━┯━━━━━━━━━━┓
┃ Feature │ MAE(Pa) │  MAPE   │ RMSE(Pa) ┃
┠─────────┼─────────┼─────────┼──────────┨
┃ Airflow │  83.55  │  9.21%  │  135.93  ┃
┃ Pres    │  6.66   │ 708.61% │  10.05   ┃
┗━━━━━━━━━┷━━━━━━━━━┷━━━━━━━━━┷━━━━━━━━━━┛

Best Parameters for LinearRegression : {}
Best Score for LinearRegression : -8.748866330485281


Training Ridge... 
--------------------------------------------------------------------------------
Fitting 5 folds for each of 1 candidates, totalling 5 fits


/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=3.12238e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.58066e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.90579e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


                  Ridge                   
┏━━━━━━━━━┯━━━━━━━━━┯━━━━━━━━━┯━━━━━━━━━━┓
┃ Feature │ MAE(Pa) │  MAPE   │ RMSE(Pa) ┃
┠─────────┼─────────┼─────────┼──────────┨
┃ Airflow │  85.48  │  9.32%  │  137.57  ┃
┃ Pres    │  6.87   │ 560.31% │  10.07   ┃
┗━━━━━━━━━┷━━━━━━━━━┷━━━━━━━━━┷━━━━━━━━━━┛

Best Parameters for Ridge : {'alpha': 0}
Best Score for Ridge : -8.795741827951925


Training Lasso... 
--------------------------------------------------------------------------------
Fitting 5 folds for each of 1 candidates, totalling 5 fits


                  Lasso                   
┏━━━━━━━━━┯━━━━━━━━━┯━━━━━━━━━┯━━━━━━━━━━┓
┃ Feature │ MAE(Pa) │  MAPE   │ RMSE(Pa) ┃
┠─────────┼─────────┼─────────┼──────────┨
┃ Airflow │  83.54  │  9.21%  │  135.92  ┃
┃ Pres    │  6.65   │ 710.25% │  10.04   ┃
┗━━━━━━━━━┷━━━━━━━━━┷━━━━━━━━━┷━━━━━━━━━━┛

Best Parameters for Lasso : {'alpha': 0.09}
Best Score for Lasso : -8.747868814758727


Training ElasticNet... 
--------------------------------------------------------------------------------
Fitting 5 folds for each of 100 candidates, totalling 500 fits


                ElasticNet                
┏━━━━━━━━━┯━━━━━━━━━┯━━━━━━━━━┯━━━━━━━━━━┓
┃ Feature │ MAE(Pa) │  MAPE   │ RMSE(Pa) ┃
┠─────────┼─────────┼─────────┼──────────┨
┃ Airflow │  83.34  │  9.28%  │  135.11  ┃
┃ Pres    │  6.69   │ 732.11% │   9.98   ┃
┗━━━━━━━━━┷━━━━━━━━━┷━━━━━━━━━┷━━━━━━━━━━┛

Best Parameters for ElasticNet : {'alpha': 0.9090909090909091, 'l1_ratio': 0.9090909090909092}
Best Score for ElasticNet : -8.819495350001704


